In [1]:
import json 
from pathlib import Path
import shutil

from wattile.entry_point import init_logging, create_input_dataframe, run_model
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, pad_full_data, input_data_split, rolling_stats
from wattile.time_processing import add_processed_time_columns
from wattile.entry_point import run_model


PROJECT_DIRECTORY = Path().resolve().parent.parent

# read configs

In [2]:

"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
#####################################################################################
# choose configs file
#####################################################################################
# with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
#     configs = json.load(f)
#####################################################################################
with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
    configs = json.load(f)
#####################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["exp_dir"] = str(exp_dir)
configs["data_dir"] = str(PROJECT_DIRECTORY / "data")

configs

{'building': 'Synthetic Site',
 'target_var': 'Synthetic Site Electricity Main Total Power',
 'start_time': '2018-01-01T00:00:00-07:00',
 'end_time': '2022-01-01T00:00:00-07:00',
 'data_time_interval_mins': 1,
 'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
  'Synthetic Weather Station Diffuse Horizontal Irradiance',
  'Synthetic Weather Station Direct Normal Irradiance',
  'Synthetic Weather Station Dry Bulb Temperature',
  'Synthetic Weather Station Global Horizontal Irradiance',
  'Synthetic Weather Station Relative Humidity',
  'Synthetic Weather Station Wind Speed'],
 'arch_version': 4,
 'exp_id': 'Debugging_for_rolling_stats',
 'arch_type': 'RNN',
 'arch_type_variant': 'lstm',
 'transformation_method': 'minmaxscale',
 'train_batch_size': 5,
 'val_batch_size': 1,
 'convert_csvs': False,
 'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir',
 'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcamp

In [3]:
# configs['feat_stats']['window_width'] = '1min'
# configs['feat_stats']['window_increment'] = '1min'

# read data

In [4]:
datatype = "complete example data" # complete example data, incomplete example data, incomplete small example data
incompleteness = False
# col_test = ['Synthetic Weather Station Direct Normal Irradiance']
col_test = []

In [5]:
import pandas as pd
from pandas.tseries.frequencies import to_offset
import numpy as np
import plotly.graph_objects as go

In [6]:
if datatype == "complete example data":
    """
    Firstly, we will read the raw data from the dataset. 
    Checkout the docs for an indepth explaination of necessary dataset structure.
    """
    data = read_dataset_from_file(configs)
    data
    
    if incompleteness == True:
        
        # data_temp = data.loc["2021-12-01":"2021-12-01" :,].copy()
        data_temp = data.copy()
        data_temp

        # adding irregular measurement intervals
        list_cols = ['Synthetic Weather Station Dew Point Temperature', 'Synthetic Weather Station Diffuse Horizontal Irradiance', 'Synthetic Weather Station Global Horizontal Irradiance']
        list_interval_mins = [3, 5, 7]
        list_timeshift_mins = [0, 3, 7]
        
        i=0
    
        for col, timestep, loffset in zip(list_cols, list_interval_mins, list_timeshift_mins):

            print("resampling and shifting column = {} with resampling timestep of {} and time-shift of {}".format(col, timestep, loffset))

            minutes = str(timestep) + "T"
            loffset = str(loffset) + "min" 
            df_temp = data_temp[col].resample(minutes).mean().copy()
            df_temp.index = df_temp.index + to_offset(loffset)
            data_temp[col] = df_temp

        # adding NaNs in random places
        fraction = 0.1
        list_index_random = data_temp.sample(frac=fraction, replace=False, random_state=1).index.tolist()
        list_column_random = pd.DataFrame(data_temp.columns).sample(frac=fraction, replace=False, random_state=2).iloc[:,0].tolist()

        i=0
        for ind in list_index_random:

            for col in list_column_random:

                #print("replacing value in index = {} and column = {} to blank".format(ind, col))
                data_temp.loc[ data_temp.index==ind , data_temp.columns==col ] = np.NAN
                
        # adding irregular/random timestamps
        def random_dates(start, end, n):

            start_u = start.value//10**9
            end_u = end.value//10**9

            return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')
        
        np.random.seed(seed=1)
        start = data_temp.index[0]
        end = data_temp.index[-1]
        n = data_temp.shape[0]
        datetime_random = random_dates(start, end, n)
        datetime_random = datetime_random.sort_values()
        datetime_random
        data_temp.index = datetime_random
        
        if col_test==[]:
            data = data_temp.copy()
        else:
            data = data_temp.loc[:, data_temp.columns.isin(col_test)]
            
elif datatype == "incomplete small example data1":

    data_test = [
        [
            "01:00:00",
            "01:01:53",
            "01:03:17",
            "01:04:02",
            "01:04:59",
            "01:05:00",
            "01:06:22",
            "01:09:46",
            "01:10:00",
            "01:11:22",
            "01:13:44",
            "01:14:26",
            "01:15:00"
        ],
        [np.nan, 1.5, 2.2, 0.9, 3.6, np.nan, 3.3, 2.3, np.nan, 1.3, 4.3, 4.1, np.nan],
        [1.0, np.nan, np.nan, np.nan, np.nan, 2.0, np.nan, np.nan, 3.0, np.nan, np.nan, np.nan, 4.0]
    ]

    data_test = pd.DataFrame(data_test).T
    data_test.columns = ['ts', 'var1', 'var2']
    data_test['var1'] = data_test['var1'].astype(float)
    data_test['var2'] = data_test['var2'].astype(float)
    data_test['ts'] = pd.to_datetime(data_test.ts)
    data = data_test.set_index('ts')
    
elif datatype == "incomplete small example data2":
    data_test = pd.read_csv(
        "../../tests/fixtures/rolling_stats_input.csv", 
        index_col=0,
    )
    data_test['var1'] = pd.to_numeric(data_test['var1'], errors='coerce')
    data_test['var2'] = pd.to_numeric(data_test['var2'], errors='coerce')
    data_test['var1'] = data_test['var1'].astype(float)
    data_test['var2'] = data_test['var2'].astype(float)
    data_test.index = pd.to_datetime(data_test.index, exact=False, utc=True)
    data = data_test[['var1','var2']]
    
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


# data processing prior to pad_full_data

In [7]:
data = correct_predictor_columns(configs, data)

In [8]:
data = correct_timestamps(configs, data)

In [9]:
data = add_processed_time_columns(data, configs)

In [10]:
data = rolling_stats(data, configs)

In [11]:
data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,DOW_binary_reg_0_min,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,105.972198


# testing pad_full_data

### set config parameters

In [12]:
# settings to put in configs
configs["feat_timelag"] = {}
configs["feat_timelag"]["lag_interval"] = "15min"
configs["feat_timelag"]["lag_count"] = 24
configs["feat_timelag"]["lag_interval_forecast"] = "0min" 

# settings to hardcode

In [13]:
def pad_full_data_old(data, configs):
    """
    Create lagged versions of exogenous variables in a DataFrame.
    Used specifically for RNN and LSTM deep learning methods.
    :param data: (DataFrame)
    :param configs: (Dict)
    :return: (DataFrame)
    """
    target = data[configs["target_var"]]
    data = data.drop(configs["target_var"], axis=1)
    data_orig = data

    # Pad the exogenous variables
    temp_holder = list()
    temp_holder.append(data_orig)
    for i in range(1, configs["window"] + 1):
        shifted = (
            data_orig.shift(i * int(configs["sequence_freq_min"]), freq="min")
            .astype("float32")
            .add_suffix("_lag{}".format(i))
        )
        temp_holder.append(shifted)
    temp_holder.reverse()
    data = pd.concat(temp_holder, axis=1)

    # If this is a linear quantile regression model (iterative)
    if configs["arch_type"] == "quantile" and configs["iterative"]:
        for i in range(0, configs["EC_future_gap_min"]):
            if i == 0:
                data[configs["target_var"]] = target
            else:
                data["{}_lag_{}".format(configs["target_var"], i)] = target.shift(-i)

        # Drop all nans
        data = data.dropna(how="any")

    # If this is a linear quantile regression model (point)
    elif configs["arch_type"] == "quantile" and not configs["iterative"]:
        # Re-append the shifted target column to the dataframe
        data[configs["target_var"]] = target.shift(-configs["EC_future_gap_min"])

        # Drop all nans
        data = data.dropna(how="any")

        # Adjust time index to match the EC values
        data.index = data.index + pd.DateOffset(minutes=(configs["EC_future_gap_min"]))

    # If this is an RNN model
    elif configs["arch_type"] == "RNN":
        # Re-append the shifted target column to the dataframe
        data[configs["target_var"]] = target.shift(-configs["EC_future_gap_min"])

        # Drop all nans
        data = data.dropna(how="any")

        # Adjust time index to match the EC values
        data.index = data.index + pd.DateOffset(minutes=(configs["EC_future_gap_min"]))

    return data

In [14]:
configs['window'] = 24
configs['sequence_freq_min'] = 15
configs['EC_future_gap_min'] = 0

pad_full_data_old(data, configs)

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,DOW_binary_reg_0_min_lag24,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 13:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,99.349998
2021-12-01 13:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-0.321439,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,99.800003
2021-12-01 13:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-0.382683,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.900002
2021-12-01 13:45:00+00:00,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-0.442289,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,101.500000
2021-12-01 14:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-0.500000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.504036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 05:45:00+00:00,16.137859,1.064227,-0.384731,47.858002,0.590813,24.799999,10.625756,-0.126199,0.992005,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,109.573402
2021-12-08 06:00:00+00:00,16.623901,-0.810933,-0.707074,47.695999,-1.170910,26.010000,10.484825,-0.061049,0.998135,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,16.985941,-0.961478,-0.831852,47.714001,-1.329346,26.340000,16.777510,0.004363,0.997859,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,102.709602


In [15]:
def pad_full_data(data, configs):
    """
    Create lagged versions of exogenous variables in a DataFrame.
    Used specifically for RNN and LSTM deep learning methods.

    :param data: (DataFrame)
    :param configs: (Dict)
    :return: (DataFrame)
    """

    # reading configuration parameters
    lag_interval = configs["feat_timelag"]["lag_interval"]
    lag_count = configs["feat_timelag"]["lag_count"]
    lag_interval_forecast = configs["feat_timelag"]["lag_interval_forecast"]
    target_var = configs["target_var"]

    # splitting predictors and target
    target = data[target_var]
    data = data.drop(target_var, axis=1)
    data_orig = data

    # padding predictors
    temp_holder = list()
    temp_holder.append(data_orig)
    for i in range(1, lag_count + 1):
        shifted = (
            data_orig.shift(freq=i * lag_interval)
            .astype("float32")
            .add_suffix("_lag{}".format(i))
        )
        temp_holder.append(shifted)
    temp_holder.reverse()
    data = pd.concat(temp_holder, axis=1)

    # If this is an RNN model
    if configs["arch_type"] == "RNN":
        # re-append the shifted target column to the dataframe
        data[target_var] = target.shift(freq="-" + lag_interval_forecast)

        # drop all nans
        data = data.dropna(how="any")

        # adjust time index to match the EC values
        data.index = data.index.shift(freq=lag_interval_forecast)

    return data

In [16]:
pad_full_data(data, configs)

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,DOW_binary_reg_0_min_lag24,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 13:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,99.349998
2021-12-01 13:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-0.321439,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,99.800003
2021-12-01 13:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-0.382683,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.900002
2021-12-01 13:45:00+00:00,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-0.442289,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,101.500000
2021-12-01 14:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-0.500000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.504036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 05:45:00+00:00,16.137859,1.064227,-0.384731,47.858002,0.590813,24.799999,10.625756,-0.126199,0.992005,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,109.573402
2021-12-08 06:00:00+00:00,16.623901,-0.810933,-0.707074,47.695999,-1.170910,26.010000,10.484825,-0.061049,0.998135,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,16.985941,-0.961478,-0.831852,47.714001,-1.329346,26.340000,16.777510,0.004363,0.997859,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,102.709602


In [17]:
def pad_full_data_s2s(data, configs):
    """
    Create lagged versions of exogenous variables in a DataFrame.
    Used specifically for Sequence to Sequence (S2S) deep learning methods.

    :param data: (DataFrame)
    :param configs: (Dict)
    :return: (DataFrame)
    """

    # reading configuration parameters
    lag_interval = configs["feat_timelag"]["lag_interval"]
    lag_count = configs["feat_timelag"]["lag_count"]
    initial_num = configs["S2S_stagger"]["initial_num"]
    secondary_num = configs["S2S_stagger"]["secondary_num"]
    decay = configs["S2S_stagger"]["decay"]
    target_var = configs["target_var"]

    target = data[target_var]
    data = data.drop(target_var, axis=1)
    data_orig = data
    # Pad the exogenous variables
    temp_holder = list()
    temp_holder.append(data_orig)
    for i in range(1, lag_count + 1):
        shifted = (
            data_orig.shift(freq=i * lag_interval)
            .astype("float32")
            .add_suffix("_lag{}".format(i))
        )
        temp_holder.append(shifted)
    temp_holder.reverse()
    data = pd.concat(temp_holder, axis=1)

    # Do fine padding for future predictions. Create a new df to preserve memory usage.
    local = pd.DataFrame()
    for i in range(0, initial_num):
        if i == 0:
            local["{}_lag_{}".format(target_var, i)] = target.shift(i)
        else:
            local["{}_lag_{}".format(target_var, i)] = target.shift(
                freq="-" + (i * lag_interval)
            )

    # Do additional coarse padding for future predictions
    for i in range(1, secondary_num + 1):
        base = initial_num
        new = int(base + decay * i)
        local["{}_lag_{}".format(target_var, base + i)] = target.shift(
            freq="-" + (new * lag_interval)
        )

    data = pd.concat([data, local], axis=1)

    # Drop all nans
    data = data.dropna(how="any")

    return data

In [18]:
pad_full_data_s2s(data, configs)

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,DOW_binary_reg_0_min_lag24,...,Synthetic Site Electricity Main Total Power_lag_62,Synthetic Site Electricity Main Total Power_lag_63,Synthetic Site Electricity Main Total Power_lag_64,Synthetic Site Electricity Main Total Power_lag_65,Synthetic Site Electricity Main Total Power_lag_66,Synthetic Site Electricity Main Total Power_lag_67,Synthetic Site Electricity Main Total Power_lag_68,Synthetic Site Electricity Main Total Power_lag_69,Synthetic Site Electricity Main Total Power_lag_70,Synthetic Site Electricity Main Total Power_lag_71
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 13:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,106.211998,106.874001,98.708000,100.105995,100.078003,102.556000,99.175995,96.543999,95.796005,100.849998
2021-12-01 13:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,106.874001,98.708000,100.105995,100.078003,102.556000,99.175995,96.543999,95.796005,100.849998,104.400002
2021-12-01 13:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,98.708000,100.105995,100.078003,102.556000,99.175995,96.543999,95.796005,100.849998,104.400002,95.938004
2021-12-01 13:45:00+00:00,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,100.105995,100.078003,102.556000,99.175995,96.543999,95.796005,100.849998,104.400002,95.938004,99.949997
2021-12-01 14:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,100.078003,102.556000,99.175995,96.543999,95.796005,100.849998,104.400002,95.938004,99.949997,98.699997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-07 12:00:00+00:00,15.855173,-1.584884,-0.187169,43.368801,-1.931602,29.110001,0.000000,0.998135,-1.608123e-16,0.0,...,104.696602,105.990005,102.918999,108.709801,103.291397,103.205399,105.448799,107.037003,111.518997,109.573402
2021-12-07 12:15:00+00:00,15.801289,-1.372290,-0.894247,42.126801,-1.967211,28.030001,0.000000,0.997859,-6.540313e-02,0.0,...,105.990005,102.918999,108.709801,103.291397,103.205399,105.448799,107.037003,111.518997,109.573402,104.668396
2021-12-07 12:30:00+00:00,15.919395,-1.401641,-0.322342,40.893799,-2.032769,30.830000,0.000000,0.991445,-1.305262e-01,0.0,...,102.918999,108.709801,103.291397,103.205399,105.448799,107.037003,111.518997,109.573402,104.668396,102.709602


In [19]:
data1 = pad_full_data_old(data, configs)
data2 = pad_full_data(data, configs)
((data1 == data2) == False).sum().sum()

0